# Notions abordées

A travers les exemples, plusieurs notions seront abordées de manières récurrentes. Ces notions sont importantes dans le langage C++. 

## Comprendre le déplacement et les références rvalue
La notion la plus complexe à appréhender est sans doute les références rvalue. Les références rvalue sont utilisées pour permettre le déplacement d'objet. En C++98, il existe une fonctions `std::swap` qui permet l'échange de valeur. Par exemple, `int a[5] = { 1, 2, 3, 4, 5}, b[5] = {}; std::swap(a,b);` permet de *déplacer* le tableau *a* vers le tableau *b*. Cette fonction peut être surchargée pour tous types, elle évite une copie et elle permet de transférer un objet qui n'a pas volonté à être dupliqué (via une relation d'amitié `friend`). A partir du C++11, l'idée de déplacer un objet a été généralisée et modernisée grâce à la notation `&&`. Une référence rvalue s'accompagne toujours de la fonction `std::move`. La fonction `std::move` s'apparente à un cast en référence rvalue. Le déplacement est généralement implémenté pour des objets proposant des services de recopie de gros objets ou pour des objets détenant une ressource unique (mutex, thread, ...) à l'aide : 
 - du constructeur de déplacement: class(class&&)
 - de l'opérateur de déplacement: class& operator=(class&&)

In [1]:
.rawInput

Using raw input


In [2]:
#include <iostream>
#include <array>
#include <memory>

class Buffer
{
private:
    using array_int5_t = std::array<int,5>;
    array_int5_t tableau;
public:   
    // Constructeur prenant une référence lvalue
    Buffer(const array_int5_t& a)
    : tableau(a)
    {
        std::cout << "copie de tableau" << std::endl;
    }
    
    // Constructeur prenant une référence rvalue
    Buffer(array_int5_t&& a)
    : tableau(std::move(a))
    {
        std::cout << "déplacement de tableau" << std::endl;
    }
    
    // Constructeur par recopie
    Buffer(const Buffer& b)
    :tableau(b.tableau)
    {
        std::cout << "recopie de buffer" << std::endl;
    }
    
    // Constructeur par déplacement
    Buffer(Buffer&& b)
    :tableau(std::move(b.tableau))
    {
        std::cout << "déplacement de buffer" << std::endl;
    }
};
             
Buffer createBuffer()
{
    return {{1,2,3,4,5}};    
}


In [3]:
.rawInput

Not using raw input


In [4]:
std::array<int,5> a{1, 2, 3, 4, 5};

std::cout << "b1 ";
Buffer b1(a);

std::cout << "b2 ";
Buffer b2({1, 2, 3, 4 , 5});

std::cout << "b3 ";
Buffer b3(createBuffer()); // RVO : return value optimization

std::cout << "b4 ";
Buffer b4(b1);

std::cout << "b5 ";
std::make_unique<Buffer>(createBuffer()); // Déplacement du tableau dans un buffer, puis du buffer dans un pointeur.

b1 copie de tableau
b2 déplacement de tableau
b3 déplacement de tableau
b4 recopie de buffer
b5 déplacement de tableau
déplacement de buffer


(std::_MakeUniq<Buffer>::__single_object) @0x7fd439090b50


__Remarque__: Un objet est déplacé seulement si celui-ci est éligible au déplacement. Par exemple, un objet `const` ou les anciennes classes implémentées en C++98 ne sont pas éligibles au déplacement. Une opération de copie est réalisée à la place.

__Remarque 2__: Un retour de fonction est toujours optimisé par le compilateur. Il ne faut pas jamais définir de rvalue sur les retours de fonction.

In [5]:
const Buffer createConstBuffer()
{
    return {{1,2,3,4,5}};    
}
std::cout << "b6 ";
std::make_unique<Buffer>(createConstBuffer());

b6 déplacement de tableau
recopie de buffer


(std::_MakeUniq<Buffer>::__single_object) @0x7fd439041da0


## Comprendre la transmission parfaite et les références universelles
L'utilisation des *template* dans des concepts de métaprogrammation a fait émerger une nouvelle problématique : la transmission parfaite des arguments. Pour palier à cet problématique, la notation `&&` pour un argument template ou une variable `auto` signifie que la variable est une référence universelle. Celle-ci indique que la variable peut être soit une référence lvalue ou soit une référence rvalue. Le type de référence est défini par l'appelant. Une référence universelle s'accompagne toujours de la fonction `std::forward`.

In [6]:
.rawInput

Using raw input


In [7]:
int *m = nullptr;

// Plusieurs fonctions de mémorisation de données en fonction de son type
void memPtr(int& a)
{
    m = &a;
}

void memVal(int a)
{}

// Une fonction template réalisant le lock de la données avant d'appeler la fonction de mémorisation
template <typename FctT, typename T>
void lockAndMem(FctT fct, T a)
{
    // appel à la fonction de lock
    fct(a);
}

// Même template mais avec une référence lvalue
template <typename FctT, typename T>
void lockAndMemRef(FctT fct, T& a)
{
    // appel à la fonction de lock
    fct(a);
}

// Même template mais avec une référence univeselle
template <typename FctT, typename T>
void lockAndMemUni(FctT fct, T&& a)
{
    // appel à la fonction de lock
    fct(std::forward<T>(a));
}

In [8]:
.rawInput

Not using raw input


In [9]:
int i = 10;
lockAndMem(&memPtr, i);
if (m != &i) std::cout << "mauvaise sauvegarde : pointeur sur une variable locale" << std::endl;

// Cela ne fonctionne pas, ajoutons une référence au template
lockAndMemRef(&memPtr, i);
if (m == &i) std::cout << "bonne sauvegarde" << std::endl;  


mauvaise sauvegarde : pointeur sur une variable locale
bonne sauvegarde


In [10]:
//lockAndMemRef(&memVal, 10); // Echoue car 10 est une rvalue.

In [11]:
// Fonctionne dans tous les cas
lockAndMemUni(&memPtr, i);
if (m == &i) std::cout << "bonne sauvegarde" << std::endl; 
lockAndMemUni(&memVal, 10);

bonne sauvegarde


(void) @0x7fd452ffbb18


## Comprendre auto

Le mot clé `auto` indique que l'on laisse le compilateur déduire le type. Le livre *Programmer efficacement en C++, ScottMeyers* donne le conseil de préférer l'utilisation de `auto` aux déclarations de type explicite. Afin de ne pas nuire à la lisibilité du code, il semble raisonnable de dire "préférer l'utilisation de `auto` aux déclarations de type explicite pour les types complexes". Une variable `auto` nécessite d'être initialisée (normal puisque le compilateur doit déduire son type). Outre l'avantage de forcer le developpeur à initialiser ses variables avec une valeur, les avantages sont :
 - de faciliter la syntaxe de types complexes comme les types à l'intérieur des conteneurs de la STL.
 - de creer des variables à partir d'expressions lambdas
 - d'éviter les erreurs de types introduisant des conversions implicites (et donc une baisse des performances)

In [12]:
.rawInput

Using raw input


In [13]:
#include <iostream>
#include <typeinfo>
#include <memory>
#include <unordered_map>
#include <functional>

#include <cxxabi.h>
std::string demangle( const char* mangled_name ) 
{
    std::size_t len = 0 ;
    int status = 0 ;
    std::unique_ptr< char, decltype(&std::free) > ptr(
                __cxxabiv1::__cxa_demangle( mangled_name, nullptr, &len, &status ), &std::free ) ;
    return ptr.get() ;
}

struct example
{
    int val;
    
    example(int a)
    : val(a)
    {}
    example(const example& o)
    : val(o.val)
    {
        std::cout << "copie" << std::endl;
    }
    ~example()
    {
        std::cout << "destruction" << std::endl;
    }
    
    bool operator==(const example& o) const
    {
        return val == o.val;
    }
};

namespace std
{
    template <> 
    struct hash<example>
    {
        size_t operator()(const example& x) const
        {
            return hash<int>()(x.val);
        }
    };
}

In [14]:
.rawInput

Not using raw input


In [15]:
auto base = 0;
auto lambda = [&base](int a, int b) { return (a + b) % base;};
std::cout << "Type lambda : " << demangle(typeid(lambda).name()) << std::endl << std::endl;

std::unordered_map<example,std::string> umap;
umap.insert(std::make_pair(21, "Hello"));
umap.insert(std::make_pair(28, "World"));
// Transmission parfaite du type contenu dans umap
for (auto&& elem : umap)
{
    std::cout << "Type du contenu dans umap : " << demangle(typeid(elem).name()) << std::endl;
}

std::cout << std::endl << "Conversion implicite dûe à une erreur de type : " << std::endl;
// le type de elem est std::pair<const example, std::string>
// de manière naïve, on aurait pu écrire : 

for (const std::pair<example, std::string>& elem : umap)
{
    // Récupération d'un pointeur sur une variable locale alors que l'on 
    // croit récupérer un pointeur sur l'élément contenu
    
    const std::pair<example, std::string>* ptr = &elem;
}

Type lambda : __cling_Un1Qu37(void*)::$_0

Type du contenu dans umap : std::pair<example const, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >
Type du contenu dans umap : std::pair<example const, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >

Conversion implicite dûe à une erreur de type : 
copie
destruction
copie
destruction


## Comprendre nullptr
Le mot clé `nullptr` permet d'indiquer un pointeur nul. A la différence de NULL qui est de type `int`, `nullptr` est de type pointeur. Il est donc indispensable d'utiliser `nullptr` pour bénéficier de la déduction des types.

In [16]:
.rawInput

Using raw input


In [17]:
template <typename FctT, typename T>
void logAndCall(FctT fct, T&& ptr)
{
    // appel à la fonction de log
    fct(std::forward<T>(ptr));
}

void fonction(int* ptr)
{}

In [18]:
.rawInput

Not using raw input


In [19]:
logAndCall(fonction, nullptr);
//logAndCall(fonction, NULL); //échoue car NULL n'est pas un pointeur

(void) @0x7fd452ffbb18


## Comprendre la génération des fonctions membres spéciales

Les fonctions membres spéciales sont les fonctions générées automatiquement par le compilateur. On en connaît déjà 4 :
 - Le constructeur par défaut
 - Le destructeur
 - Le constructeur de recopie
 - L'opérateur de copie.

Le C++11 ajoute au 4 premiers :
 - Le constructeur de déplacement
 - L'opérateur de déplacement

Le comportement par défaut pour ces deux opérations est un déplacement membre à membre de la classe.
Cependant, les règles de génération se sont un peu plus durcies par rapport à C++98 pour ces deux nouveaux. Pour les 4 premiers, la génération est indépendante, c'est à dire que si on déclare un destructeur, le constructeur de recopie et l'opérateur de copie sont quand même générés. Souvent, ce comportement n'est pas souhaité car si l'on déclare un destructeur, c'est qu'une ressource est détenue par la classe et nécessite une implémentation spéciale pour sa copie. Ce qui à engendré la règle de codage que tous le monde connaît qui est d'implémenter le destructeur, le constructeur de recopie et l'opérateur de copie lorsque la classe alloue une ressource.
Avec les deux nouveaux, le problème ne se pose pas car elle ne sont pas générés si l'un des points suivants est vérifié :
 - Le destructeur est déclaré
 - Le constructeur de recopie ou l'opérateur de copie est déclaré
 - Une opération de déplacement est déclaré

Pour le dernier point, cela s'explique par le fait que s'il existe une implémentation pour le constructeur de déplacement, alors la génération du comportement par défaut pour l'opérateur de déplacement ne convient pas. Le raisonnement inverse est identique. 

De plus, si une opération de déplacement est déclarée, les opérations de recopie ne sont pas générées.

Ces règles sont importantes. Prenons l'exemple que l'on crée un destructeur pour pouvoir tracer les appels, cette pratique va avoir un effet indésirable car à la place d'utiliser les opérations de déplacement parfaitement valides, le compilateur utilise les opérations de recopie pouvant être beaucoup plus coûteuses. Pour éviter cela, il est possible d'utiliser le mot clé `default` pour signifier que le comportement de déplacement par défaut est valide.

Une autre utilisation du mot clé `default` est l'utilisation pour le destructeur virtuel des classes de base. En effet, une règle du C++ impose de déclarer un destructeur virtuel de la classe de base pour permettre la désallocation complète de l'objet lors de l'utilisation de polymorphisme dynamique. Ce destructeur ne fait rien et le comportement par défaut est adapté au besoin.

In [20]:
class BufferCopie
{
public:
    BufferCopie()
    {}

    BufferCopie(const BufferCopie& b)
    {
        std::cout << "recopie de buffer" << std::endl;
    }
};

class BufferDeplacement
{
public:
    BufferDeplacement()
    {}

    BufferDeplacement(const BufferDeplacement& b)
    {
        std::cout << "recopie de buffer" << std::endl;
    }
    
    BufferDeplacement(BufferDeplacement&& b) = default;
};
// Pas de génération du constructeur de déplacement car le constructeur de recopie est défini.
// Appel au constructeur de recopie
std::cout << "b7 ";
std::make_unique<BufferCopie>(BufferCopie());

// Le constructeur de recopie est défini et le constructeur de déplacement est défini avec le comportement par défaut.
// Appel au constructeur de déplacement
std::cout << "b8 ";
std::make_unique<BufferDeplacement>(BufferDeplacement());

// Définition d'une classe interface
class INotifier
{
public:
    // Déclaration du destructeur virtuel pour l'appel au destructeur de la classe héritée lors de l'utilisation du 
    // polymorphisme dynamique
    virtual ~INotifier() = default;
    virtual Notify() = 0;
};

b7 recopie de buffer
b8 

(std::_MakeUniq<BufferDeplacement>::__single_object) @0x7fd439ccb810


## Comprendre les fonctions supprimés
La génération des fonctions membres spéciales est parfois non souhaitée. Par exemple, la copie de classes de gestion d'entrées/sorties n'est pas vraiment un concept définissable : le port d'entrée doit-il être réinitialisé ? ou partagé ? Il est plus simple de d'interdire la copie de telle classe. Le mot-clé `delete` permet d'indiquer qu'une fonction est interdite. 

In [4]:
class CopieInterdite
{
public:
    CopieInterdite()
    {}
    CopieInterdite(const CopieInterdite&) = delete;
    CopieInterdite& operator=(const CopieInterdite&) = delete;
};

CopieInterdite A, B;
//CopieInterdite C(A);
//B = A;

## Comprendre les fonctions de substitutions

## Comprendre constexpr

## Comprendre noexcept

## Comprendre using

## Comprendre decltype

## Comprendre enum class

## Comprendre les initializers